# Milestone 2 in Data Science
## Fake news Predictor

### Group 8
*Lykke Laura Sørensen (ltm712)* <br>
*Daniel Falk Knudsen (gkb913)* <br>
*Jeppe Ram Pedersen (lxd520)*

In [1]:
# importing the relevant libraries
!pip install pandas
import pandas as pd
!pip install sqlalchemy
import sqlalchemy

### Task 1
The first task is to demonstrate that you have a working database containing the FakeNewsCorpus dataset.<br>
Explain your choice of schema design.<br>
You have been working on this task on a small subset of the data during the TA-sessions.<br>
For this milestone, demonstrate that your database contains a larger number of rows (e.g. one million - or however many you can reasonably work with on your available hardware), and that it supports simple queries.

In [2]:
# define relevant path for file
dataForMilestone2 = '../250t-raw.csv'

# read the csv-file and load into dataframe
dataRaw = pd.read_csv(dataForMilestone2)

C:\Users\Lykke\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


We make a preliminary investigation of our data, in order to filter out irrelevant tables.<br>

In [25]:
def fieldLengthPrinter (string):
  print(column, len(dataRaw[string].value_counts()))

for column in dataRaw:
  fieldLengthPrinter (column)

display(dataRaw[:3])

Unnamed: 0 20007
id 250513
domain 356
type 12
url 250511
content 178223
scraped_at 2
inserted_at 1
updated_at 1
title 222069
authors 29030
keywords 0
meta_keywords 30093
meta_description 65459
tags 41991
summary 0
source 0


,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
2,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN


We observe the following:
1. The columns keywords, summary and source are not used and is not useful going forward.
2. The first column is a remnant of old indeces and is not useful going forward.
3. The column id is not clear enough as to which id is meant and should be renamed. Same applies to domiains, types and authors.
4. The columns scraped_at, inserted_at, and updated_at only have 1 or 2 unique values. We will however keep these columns, since the will increase as time goes on and more articles are added.

With these observations in mind, we make a modified dataframe containing all relevant information.<br>
A modified dataframe is arguably unneccessary, but we do not want to manipulate our original input.

In [4]:
# Modified dataframe used as source for other dataframes
dataMod = dataRaw
dataMod = dataMod[['id','domain','type','url','content','scraped_at','inserted_at','updated_at','title','authors','meta_keywords','meta_description','tags']]
dataMod = dataMod.rename(columns={'id':'ArticleID','domain':'DomainName','type': 'TypeName','authors':'AuthorGroupMembers'})
display(dataMod)

,ArticleID,DomainName,TypeName,url,content,scraped_at,inserted_at,updated_at,title,AuthorGroupMembers,meta_keywords,meta_description,tags
0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,[''],THE UNIVERSE ceases to exist when we are not l...,NaN
1,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",[''],NaN,NaN
2,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",[''],NaN,NaN
3,10,barenakedislam.com,hate,http://barenakedislam.com/2017/12/27/following...,The Central American nation and six other stat...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Following Guatemala’s decision to move its emb...,"Pamela Jeffersons, Cleavis Nowell, Cleavisnowe...",[''],NaN,NaN
4,14,blackagendareport.com,unreliable,https://blackagendareport.com/articlelist/inte...,"“When the police finally left the campus, arou...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Black Agenda Report,"Ann Garrison, Bar Contributor, Ken Morgan, Jem...",[''],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250524,1170100,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ANK...,Raw content\n\nCONFIDENTIAL PAGE 01 ANKARA 019...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ANKARA01989,NaN,[''],NaN,View Tags
250525,1170103,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974JAK...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974JAKART02409,NaN,[''],NaN,View Tags
250526,1170110,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ROM...,Raw content\n\nLIMITED OFFICIAL USE PAGE 01 RO...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ROME16372,NaN,[''],NaN,View Tags
250527,1170111,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976STA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976STATE222598,NaN,[''],NaN,View Tags


With the modified dataframe, we can extract the relevant dataframes for our tables.<br>
In our schema, we decided to focus on decoupling articles, authors, types and domains.<br>
We excluded a lot of data irrelevant to this focus - this was done since time would not allow for more, it is not an optimal methodology.<br>
<br>
NB: It would have been optimal to decouple seperate authors from the "authors" column, since every field is a csv-list of contributing authors - time did not permit this. (This would also have been the case for most data, that was excluded.) <br>
This could be done using the unest-function. 

### The design is the following:
We want a main table, where we have data unique to each article. This includes articleID, url, content and title.<br>
The term auxillary categories is in the following used to mean authors, types and domains.<br>
Since we have many-to-one between articles and auxillary categories, we want to decouple them from each other.<br>
In order to achieve this, we want separate tables for each auxillary category and to each element in each category give an ID.<br>
Since we do not want these auxillary IDs in our main table, we want a connector table for each auxillary category, where ArticleID and auxillary ID is paired.<br>
This is done, to avoid problems with deletions (although this will not be a problem for this particular dataset.)<br>
### To summarize:
- We want a main table for articles.
- We want 3 auxillary categories: authors, types, and domains.
- We want a table for each auxillary category with ID and name for each element.
- We want tables connecting articles to relevant auxillary categories by matching articleID with auxillary category ID

In [5]:
# Dataframe for main table
dataArticles = dataMod[['ArticleID','url','content','title']]

# Dataframes for auxillary tables
dataAuthors = dataMod[['AuthorGroupMembers']].drop_duplicates().reset_index(drop=True)
dataAuthors['AuthorGroupID'] = dataAuthors.index

dataType = dataMod[['TypeName']].drop_duplicates().reset_index(drop=True)
dataType['TypeID'] = dataType.index

dataDomain = (dataMod[['DomainName']].drop_duplicates().reset_index(drop=True))
dataDomain['DomainID'] = dataDomain.index

# Dataframes for connecting tables
modRawAut = pd.merge(dataAuthors, dataMod, on=['AuthorGroupMembers'], how='outer')
ArtIDToAutID = modRawAut[['ArticleID','AuthorGroupID']]

modRawType = pd.merge(dataType, dataMod, on=['TypeName'], how='outer')
ArtIDToTypeID = modRawType[['ArticleID','TypeID']]

modRawDomain = pd.merge(dataDomain, dataMod, on=['DomainName'], how='outer')
ArtIDToDomainID = modRawDomain[['ArticleID','DomainID']]

After having made suitable dataframes, we now put them into a tables.<br>
We found establishing connection to pg4Admin and the like led to unnecessary frustrations, hence we use sqlalchemy to make an sqlite server in memory.

In [6]:
# Making Dataframes into tables accessible with SQL
engine = sqlalchemy.create_engine('sqlite://', echo=False)

dataMod.to_sql('mod', con=engine, index=False)
dataArticles.to_sql('Articles', con=engine, index=False)
dataAuthors.to_sql('AuthorGroups', con=engine, index=False)
dataType.to_sql('Types', con=engine, index=False)
dataDomain.to_sql('Domains', con=engine, index=False)
ArtIDToAutID.to_sql('Art2Aut', con=engine, index=False)
ArtIDToTypeID.to_sql('Art2Type', con=engine, index=False)
ArtIDToDomainID.to_sql('Art2Dom', con=engine, index=False)

We now only need to demonstrate, that we can make simple queries.<br>
The following queries illustrate this:

In [7]:
display(engine.execute("SELECT * FROM Art2Dom WHERE DomainID=2").fetchall())
display(engine.execute("SELECT url FROM Articles").fetchall())
display(engine.execute("SELECT * FROM AuthorGroups WHERE AuthorGroupMembers IS NULL").fetchall())

[('14', 2),
 ('54', 2),
 ('64', 2),
 ('134', 2),
 ('224', 2),
 ('279', 2),
 ('282', 2),
 ('472', 2),
 ('537', 2),
 ('654', 2),
 ('771', 2),
 ('772', 2),
 ('844', 2),
 ('899', 2),
 ('1098', 2),
 ('1191', 2),
 ('1401', 2),
 ('1405', 2),
 ('18098', 2),
 ('79909', 2),
 ('80005', 2),
 ('80013', 2),
 ('80037', 2),
 ('80041', 2),
 ('80043', 2),
 ('80062', 2),
 ('139383', 2),
 ('466520', 2),
 ('484530', 2),
 ('593059', 2),
 ('593063', 2),
 ('593065', 2),
 ('593074', 2),
 ('593079', 2),
 ('593088', 2),
 ('593089', 2),
 ('593094', 2),
 ('593095', 2),
 ('593101', 2),
 ('593105', 2),
 ('593110', 2),
 ('593111', 2),
 ('593112', 2),
 ('593116', 2),
 ('593123', 2),
 ('593147', 2),
 ('593149', 2),
 ('593153', 2),
 ('593179', 2),
 ('593181', 2),
 ('593207', 2),
 ('593212', 2),
 ('593221', 2),
 ('593225', 2),
 ('593245', 2),
 ('658751', 2),
 ('658756', 2),
 ('691233', 2),
 ('725020', 2),
 ('810033', 2),
 ('836825', 2),
 ('836826', 2),
 ('841600', 2),
 ('867231', 2),
 ('904145', 2),
 ('908356', 2),
 ('94

[('https://www.express.co.uk/news/science/738402/life-an-ILLUSION-reality-does-not-exist-if-you-are-not-looking-at-it',),
 ('http://barenakedislam.com/category/donald-trump/page/2/',),
 ('http://barenakedislam.com/2017/12/24/more-winning-israeli-intelligence-source-debkafile-confirms-the-trump-administration-to-cut-off-ties-with-the-palestinians-which-means-no-peace-plan-no-more-financial-aid/',),
 ('http://barenakedislam.com/2017/12/27/following-guatemalas-decision-to-move-its-embassy-to-jerusalem-the-capital-of-israel-10-more-countries-are-also-considering-the-move/',),
 ('https://blackagendareport.com/articlelist/international/articlelist',),
 ('http://blackgenocide.org/speaking_request.html',),
 ('http://blackgenocide.org/archived_articles/oppose.html',),
 ('http://bipartisanreport.com/cdn-cgi/l/email-protection',),
 ('http://awm.com/man-goes-to-throw-away-christmas-tree-doesnt-realize-dog-is-following-him-ends-in-hilarity/',),
 ('https://www.blacklistednews.com/Egypt%E2%80%99s_rac

[(None, 5)]

### Task 2
List the relations you have created in your database. For each relation:

1.   list its attributes
2.   list its functional dependencies.
3.   list all the primary keys.

Is each relation in BCNF form?<br>
If not, show how to transform the tables in BCNF and explain why it might be better (or not) to use the BCNF relations in your database.

The BCNF can be summarised to the following rules:
- NF1: Atomized values in cells
- NF2: No non-primary key is dependedent on any proper subset of candidate keys
- NF3: No embedded tables
- NF3.5: Primary key should be a single attribute

### Relations:
#### Articles
- Attributes:
    - ArticleID
    - url
    - content
    - title
- Functional Dependencies:
    - ArticleID -> url
    - ArticleID -> content
    - ArticleID -> title
- Primary Keys:
    - ArticleID
- BCNF compliant.
    
#### AuthorGroups
- Attributes:
    - AuthorGroupID
    - AuthorGroupMembers
- Functional Dependencies:
    - AuthorGroupID -> AuthorGroupMembers
- Primary Keys
    - AuthorGroupID
- Not NF1 compliant due to AuthorGroupMembers being lists. To remedy: Make new relation mapping AuthorGroupID to AuthorID, seperate make new relation where AuthorID is related to AuthorName.

#### Types
- Attributes:
    - TypeID
    - TypeName
- Functional Dependencies:
    - TypeID -> TypeName
- Primary Keys:
    - TypeID
- BCNF compliant

#### Domains
- Attributes:
    - DomainID
    - DomainName
- Functional Dependencies:
    - DomainID -> DomainName
- Primary Keys
    - DomainID
- BCNF compliant

#### Art2Aut
- Attributes:
    - ArticleID
    - AuthorGroupID
- Functional Dependencies:
    - ArticleID -> AuthorGroupID
- Primary Keys
    - (ArticleID,AuthorGroupID)
- BCNF compliant

#### Art2Type
- Attributes:
    - ArticleID
    - TypeID
- Functional Dependencies:
- Primary Keys
    - (ArticleID,TypeID)
- BCNF compliant

#### Art2Dom
- Attributes:
    - ArticleID
    - DomainID
- Functional Dependencies:
- Primary Keys
    - (ArticleID,DomainID)
- BCNF compliant

### Task 3

Once your database is loaded, you can start issuing queries to better understand the characteristics of the data.<br>
Formulate the following queries in the database languages requested (in the square brackets following each item) and briefly discuss what you observe when you execute them over your database:   

1.   List the domains of news articles of reliable type and scraped at or after January 15, 2018. NOTE: Do not include duplicate domains in your answer. \[Languages: relational algebra and SQL]
2.   List the name(s) of the most prolific author(s) of news articles of fake type. An author is among the most prolific if it has authored as many or more fake news articles as any other author in the dataset. \[Languages: extended relational algebra and SQL]
3.   Count the pairs of article IDs that exhibit the exact same set of meta-keywords, but only return the pairs where the set of meta-keywords is not empty. \[Language: SQL]

#### Task 3.1
Since none of our tables include scraped_at, we use the modified table

Task 3.1:
P_domain(S_type=reliable,(scraped_at)_20180115(mod))

In [8]:
print("Task 3.1: List of the domains of the reliable articles scraped after 2018-01-15:")
engine.execute("SELECT DISTINCT DomainName FROM mod WHERE TypeName='reliable' AND scraped_at>'2018-01-15'").fetchall()

Task 3.1: List of the domains of the reliable articles scraped after 2018-01-15:


[('christianpost.com',), ('consortiumnews.com',), ('nutritionfacts.org',)]

#### Task 3.2
Task 3.2: P_author((S_type=fake) join (S_count(Authors)=max(count(Authors))))

In [9]:
engine.execute("SELECT DISTINCT AuthorGroupMembers, COUNT(AuthorGroupMembers) AS AuthorCount FROM mod WHERE TypeName='fake' GROUP BY AuthorGroupMembers HAVING AuthorCount>=AuthorCount ORDER BY AuthorCount DESC").fetchall()[0:2]

[('John Rolls', 280), ('Zacks Investment Research', 237)]

#### Task 3.3
Unable to fulfill the assignment completely due to inadequate cleaning of data.

In [10]:
task3 = engine.execute("SELECT meta_keywords FROM mod WHERE LENGTH(meta_keywords)>4 ORDER BY meta_keywords ASC")
task3.fetchall()

[('["\'Early image of Jesus\' unearthed in Egypt by Spanish archeologists", \'Spain\', \'Egypt\', \'Cairo\', \'Jesus Christ\', \'Archeology\', \'discovery\']',),
 ('["\'Foreign Men\' Molest And Rape 40 Girls At Swedish Music Festival"]',),
 ('["\'Global demand for gas will rise\' - Qatar minister", \'Russia\', \'World\', \'gas supplies\', \'Qatar\', "gas exporters\' forum", \'gas exchange\', \'politicians\']',),
 ('["\'High Capacity\' Magazine Ban", \'Ammunition Background Checks\', \'Gavin Newsom\', \'gun control\', \'2nd Amendment\', \'Breitbart California\', \'gavin newsom\']',),
 ('["\'Home-grown\'", \'Abdulazeez\', \'Chattanooga\', \'FBI\', \'Big Government\', \'Chattanooga\']',),
 ('["\'I believe that British law can work\' - Litvinenko’s widow", \'Russia\', \'Litvinenko case\', \'Great Britain\', \'investigation\', \'Alexander Litvinenko\']',),
 ('["\'It\'s a tragedy Mrs. Thatcher ever came to power\' - Ken Livingstone", \'World\', \'Margaret Thatcher\', \'Great Britain\', \'UK\

### Task 4.

Now that we have our data in a database, let’s revisit the “interesting observations” task from Milestone 1 - but now using queries to the database.<br>
The idea is to write database queries (e.g. using GROUP BY and COUNT) that explore features of the data set that are relevant to the fake news prediction task: outliers, artefacts.<br>
It’s OK to investigate the same issues as in Milestone 1 (now using database queries) - but you are also very welcome to come up with completely new queries.<br>
You should write at least 3 such queries.

In [11]:
numOfDomains = engine.execute("SELECT COUNT(DomainID) FROM Domains WHERE DomainName IS NOT NULL").fetchall()[0][0]
print("There are %d domains" % numOfDomains)

There are 356 domains


In [12]:
numOfDomainTypeCombinations = engine.execute("SELECT COUNT(*) FROM (SELECT DISTINCT DomainID, TypeID FROM Art2Dom NATURAL JOIN Art2Type)").fetchall()[0][0]
print("There are %d domains and %d domain-type combinations" %(numOfDomains,numOfDomainTypeCombinations))

There are 356 domains and 357 domain-type combinations


It is clear to see that the dataset is biased since every domain only has one type of news, except in one case where a domain has two types. 

In [13]:
numOfAuthorGroups = engine.execute("SELECT COUNT(AuthorGroupID) FROM AuthorGroups WHERE AuthorGroupMembers IS NOT NULL").fetchall()[0][0]

numOfDomainAuthorsCombinations = engine.execute("SELECT COUNT(*) FROM (SELECT DISTINCT DomainID, AuthorGroupID FROM Art2Dom NATURAL JOIN Art2Aut)").fetchall()[0][0]

print("There are %d Author groups and %d domain-Author group combinations" % (numOfAuthorGroups,numOfDomainAuthorsCombinations))

There are 29030 Author groups and 29565 domain-Author group combinations


It is possible that every group of authors only write for a limited number of domains where at least 28495 write for a single domain. (Due to the current state of the database however, this will be difficult to ascertain since the table AuthorGroups is not NF1 compliant.)

In [14]:
engine.execute("SELECT DomainName, TypeName, COUNT(ArticleID) FROM (SELECT * FROM Art2Dom NATURAL JOIN Art2Type NATURAL JOIN Domains NATURAL JOIN Types) GROUP BY DomainID ORDER BY COUNT(ArticleID) DESC").fetchall()

[('dailykos.com', 'political', 40387),
 ('wikileaks.org', 'unreliable', 33889),
 ('beforeitsnews.com', 'fake', 29729),
 ('express.co.uk', 'rumor', 11596),
 ('wikispooks.com', 'conspiracy', 8492),
 ('abovetopsecret.com', 'conspiracy', 8320),
 ('sputniknews.com', 'bias', 6864),
 ('unz.com', 'bias', 6018),
 ('shadowproof.com', 'unknown', 5151),
 ('breitbart.com', 'political', 5059),
 ('www.freedomsphoenix.com', None, 3417),
 ('dailycaller.com', 'political', 3112),
 ('us.blastingnews.com', 'satire', 3057),
 ('redstate.com', 'political', 2785),
 ('rawstory.com', 'political', 2734),
 ('investmentwatchblog.com', 'bias', 2712),
 ('teapartytribune.com', 'unknown', 2702),
 ('naturalnews.com', 'junksci', 2507),
 ('katehon.com', 'bias', 2454),
 ('weeklystandard.com', 'political', 2317),
 ('zerohedge.com', 'conspiracy', 2061),
 ('washingtonexaminer.com', 'political', 1721),
 ('lifenews.com', 'bias', 1655),
 ('lifesitenews.com', 'clickbait', 1640),
 ('liberalamerica.org', 'clickbait', 1608),
 ('dark

We see that three domains account for approximately 2/5 of all entries. Since, we have shown that only one domain publish two types of articles the tendencies will be heavily skewed towards these three types: political, unreliable and fake.